In [1]:
import os
import librosa
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression, Perceptron
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.preprocessing import StandardScaler
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
import joblib

# Step 1: Set the path to the local dataset folder
desktop_path = os.path.expanduser("~/Desktop/22F-3625-DS-A-04/deepfake_detection_dataset_urdu")  # Portable path to Desktop folder

# Step 2: Define subfolder paths
bonafide_dir = os.path.join(desktop_path, "Bonafide")
spoofed_tacotron_dir = os.path.join(desktop_path, "Spoofed_Tacotron")
spoofed_tts_dir = os.path.join(desktop_path, "Spoofed_TTS")

# Step 3: Feature Extraction Function (MFCCs)
def extract_mfcc(file_path, max_len=300):
    try:
        y, sr = librosa.load(file_path, sr=None)
        mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
        if mfcc.shape[1] < max_len:
            pad_width = max_len - mfcc.shape[1]
            mfcc = np.pad(mfcc, pad_width=((0, 0), (0, pad_width)), mode='constant')
        else:
            mfcc = mfcc[:, :max_len]
        return mfcc.flatten()
    except Exception as e:
        print(f"Error processing {file_path}: {e}")
        return None


In [2]:

# Step 4: Extract Features & Labels
X, y = [], []
label_map = {"bonafide": 0, "spoof": 1}

# Helper function to process all .wav files in a directory (for Part 1 and Part 2)
def process_wav_files(directory, label, desc):
    if not os.path.exists(directory):
        print(f"Directory not found: {directory}")
        return

    # Get list of .wav files in the directory
    wav_files = [f for f in os.listdir(directory) if f.endswith(".wav")]

    # Process each .wav file with progress bar
    for wav_file in tqdm(wav_files, desc=desc):
        file_path = os.path.join(directory, wav_file)
        features = extract_mfcc(file_path)
        if features is not None:
            X.append(features)
            y.append(label)

# Helper function to process all .wav files in a Speaker_XX folder (for Bonafide)
def process_bonafide_speaker_folder(speaker_dir, desc):
    if not os.path.exists(speaker_dir):
        print(f"Directory not found: {speaker_dir}")
        return

    # Get list of Speaker_XX folders
    speaker_folders = [f for f in os.listdir(speaker_dir) if f.startswith("Speaker_") and os.path.isdir(os.path.join(speaker_dir, f))]

    # Process each Speaker_XX folder
    for speaker_folder in tqdm(speaker_folders, desc=desc):
        speaker_path = os.path.join(speaker_dir, speaker_folder)
        # Process Part 1 and Part 2 within each Speaker_XX folder
        part1_dir = os.path.join(speaker_path, "Part 1")
        part2_dir = os.path.join(speaker_path, "Part 2")
        process_wav_files(part1_dir, label_map["bonafide"], f"Processing {speaker_folder} Part 1")
        process_wav_files(part2_dir, label_map["bonafide"], f"Processing {speaker_folder} Part 2")

# Helper function to process all .wav files in a Speaker_XX folder (for Spoofed_*)
def process_spoofed_speaker_folder(speaker_dir, label, desc):
    if not os.path.exists(speaker_dir):
        print(f"Directory not found: {speaker_dir}")
        return

    # Get list of Speaker_XX folders
    speaker_folders = [f for f in os.listdir(speaker_dir) if f.startswith("Speaker_") and os.path.isdir(os.path.join(speaker_dir, f))]

    # Process each Speaker_XX folder
    for speaker_folder in tqdm(speaker_folders, desc=desc):
        speaker_path = os.path.join(speaker_dir, speaker_folder)
        # Get list of .wav files in the Speaker_XX folder
        wav_files = [f for f in os.listdir(speaker_path) if f.endswith(".wav")]
        for wav_file in wav_files:
            file_path = os.path.join(speaker_path, wav_file)
            features = extract_mfcc(file_path)
            if features is not None:
                X.append(features)
                y.append(label)

# Process Bonafide files (Speaker_XX/Part 1 and Part 2)
process_bonafide_speaker_folder(bonafide_dir, "Processing Bonafide")

# Process Spoofed_Tacotron files
process_spoofed_speaker_folder(spoofed_tacotron_dir, label_map["spoof"], "Processing Spoofed_Tacotron")

# Process Spoofed_TTS files
process_spoofed_speaker_folder(spoofed_tts_dir, label_map["spoof"], "Processing Spoofed_TTS")

X = np.array(X)
y = np.array(y)

# Step 5: Check if data is loaded
if len(X) == 0 or len(y) == 0:
    raise ValueError("No valid audio files were processed. Check file paths and formats.")

# Step 6: Scale Features
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Step 7: Split Data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Step 8: Evaluation Function
def evaluate_model(name, model, X_test, y_test, probas=None):
    y_pred = model.predict(X_test)
    print(f"\n{name} Evaluation:")
    print("Accuracy:", accuracy_score(y_test, y_pred))
    print("Precision:", precision_score(y_test, y_pred))
    print("Recall:", recall_score(y_test, y_pred))
    print("F1 Score:", f1_score(y_test, y_pred))
    if probas is not None:
        print("AUC-ROC:", roc_auc_score(y_test, probas))

# Step 9: Train SVM
svm_model = SVC(probability=True)
svm_model.fit(X_train, y_train)
evaluate_model("SVM", svm_model, X_test, y_test, svm_model.predict_proba(X_test)[:, 1])

# Step 10: Train Logistic Regression
lr_model = LogisticRegression(max_iter=1000)
lr_model.fit(X_train, y_train)
evaluate_model("Logistic Regression", lr_model, X_test, y_test, lr_model.predict_proba(X_test)[:, 1])


Directory not found: /root/Desktop/22F-3625-DS-A-04/deepfake_detection_dataset_urdu/Bonafide
Directory not found: /root/Desktop/22F-3625-DS-A-04/deepfake_detection_dataset_urdu/Spoofed_Tacotron
Directory not found: /root/Desktop/22F-3625-DS-A-04/deepfake_detection_dataset_urdu/Spoofed_TTS


ValueError: No valid audio files were processed. Check file paths and formats.

In [ ]:
# Step 11: Train Perceptron
perceptron_model = Perceptron(max_iter=1000)
perceptron_model.fit(X_train, y_train)
evaluate_model("Perceptron", perceptron_model, X_test, y_test)

# Step 12: Define DNN Class
class DNN(nn.Module):
    def __init__(self, input_dim):
        super(DNN, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 256),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.net(x)

# Step 13: Prepare Data for PyTorch
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.reshape(-1, 1), dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.reshape(-1, 1), dtype=torch.float32)

# Step 14: Train DNN
dnn_model = DNN(X.shape[1])
criterion = nn.BCELoss()
optimizer = optim.Adam(dnn_model.parameters(), lr=0.001)

print("\nTraining DNN:")
for epoch in range(10):
    dnn_model.train()
    outputs = dnn_model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    print(f"Epoch {epoch+1}, Loss: {loss.item():.4f}")

# Step 15: Evaluate DNN
dnn_model.eval()
with torch.no_grad():
    y_pred_probs = dnn_model(X_test_tensor).numpy().flatten()
    y_pred = (y_pred_probs > 0.5).astype(int)

print("\nDNN Evaluation:")
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Precision:", precision_score(y_test, y_pred))
print("Recall:", recall_score(y_test, y_pred))
print("F1 Score:", f1_score(y_test, y_pred))
print("AUC-ROC:", roc_auc_score(y_test, y_pred_probs))

# Step 16: Save Models
joblib.dump(lr_model, "lr_audio_model.joblib")
joblib.dump(svm_model, "svm_audio_model.joblib")
joblib.dump(perceptron_model, "perceptron_audio_model.joblib")
torch.save(dnn_model.state_dict(), "dnn_audio_model.pth")
joblib.dump(scaler, "audio_scaler.joblib")
print("\nModels saved successfully for Part 1!")